In [1]:
library(igraph)
library(dplyr)
library(tibble)


Attaching package: ‘igraph’


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union



Attaching package: ‘dplyr’


The following objects are masked from ‘package:igraph’:

    as_data_frame, groups, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘tibble’


The following object is masked from ‘package:igraph’:

    as_data_frame




In [2]:
.libPaths()

[1] "/Users/mnajm/miniconda3/envs/r_venv_conda/lib/R/library"

# 1. CF network

## 1.1 Loading the CF network

In [3]:
CF_PPI_network.pruned.interactions <- 
  read.table(file = "../CFnetwork/data/kegg_diff_pathways_network/diff_kegg_pathways_with_CFTR_interactors_PPI_direct_pruned_interactions_df.txt",
             sep = "\t",
             header = T,
             check.names = F)

CF_PPI_network.pruned.nodes <- 
  read.table(file = "../CFnetwork/data/kegg_diff_pathways_network/diff_kegg_pathways_with_CFTR_interactors_PPI_direct_pruned_nodes_df.txt",
             sep = "\t",
             header = T,
             check.names = F)

## 1.2 As an igraph object

In [4]:
source("../CFnetwork/scripts/pathways_to_network/network_utils.R")

In [5]:
CF_PPI_network.pruned <- new("PPI_network",
                             interactions=CF_PPI_network.pruned.interactions,
                             nodes=CF_PPI_network.pruned.nodes)
CF_PPI_network.pruned.with_CFTR <- CF_PPI_network.pruned

In [6]:
dim(CF_PPI_network.pruned.with_CFTR@interactions)

[1] 555  28

In [7]:
dim(CF_PPI_network.pruned@nodes)

[1] 339  22

## 1.3 Remove CFTR from the network and its indirect interactors

In [8]:
# CFTR_indirect_interactors <- CF_PPI_network.pruned.nodes[which(is.na(CF_PPI_network.pruned.nodes$sum)),
#                                                          "Symbol"]

# CF_PPI_network.pruned@nodes <- CF_PPI_network.pruned@nodes[which(!CF_PPI_network.pruned@nodes$Symbol %in% c("CFTR", CFTR_indirect_interactors)),]
# CF_PPI_network.pruned@interactions <- CF_PPI_network.pruned@interactions[which(!CF_PPI_network.pruned@interactions$genesymbol_source %in% c("CFTR", CFTR_indirect_interactors) &
#                                                                                  !CF_PPI_network.pruned@interactions$genesymbol_target %in% c("CFTR", CFTR_indirect_interactors)),]

In [9]:
CF_PPI_network.pruned@nodes <- CF_PPI_network.pruned@nodes[which(CF_PPI_network.pruned@nodes$Symbol!="CFTR"),]
CF_PPI_network.pruned@interactions <- CF_PPI_network.pruned@interactions[which(CF_PPI_network.pruned@interactions$genesymbol_source!="CFTR" &
                                                                                 CF_PPI_network.pruned@interactions$genesymbol_target!="CFTR"),]

In [10]:
dim(CF_PPI_network.pruned@nodes)

[1] 338  22

In [11]:
dim(CF_PPI_network.pruned@interactions)

[1] 543  28

## 1.4 Binding interactions into two directed interactions in both directions

In [12]:
# Pb of binding interactions
## Non binding
CF_PPI_network.pruned.interactions.non_binding <- 
  CF_PPI_network.pruned@interactions[which(!(CF_PPI_network.pruned@interactions$effect %in% c("binding/association"))),]
CF_PPI_network.pruned.interactions.non_binding <- 
  CF_PPI_network.pruned.interactions.non_binding[,c("genesymbol_source",
                                                           "genesymbol_target")]
colnames(CF_PPI_network.pruned.interactions.non_binding) <- c("from", 
                                                                     "to")

# ## Dissociation
CF_PPI_network.pruned.interactions.binding <- 
  CF_PPI_network.pruned@interactions[which(CF_PPI_network.pruned@interactions$effect %in% c("binding/association")),]
# Both directions for binding interactions
## one direction
CF_PPI_network.pruned.interactions.binding.one_direction <- 
  CF_PPI_network.pruned.interactions.binding[,c("genesymbol_source",
                                                       "genesymbol_target")]
colnames(CF_PPI_network.pruned.interactions.binding.one_direction) <- c("from", 
                                                                               "to")
## other direction
CF_PPI_network.pruned.interactions.binding.other_direction <- 
  CF_PPI_network.pruned.interactions.binding[,c("genesymbol_target", 
                                                       "genesymbol_source")]
colnames(CF_PPI_network.pruned.interactions.binding.other_direction) <- c("from", 
                                                                                 "to")
## both directions
CF_PPI_network.pruned.interactions.binding.both_directions <- 
  rbind(CF_PPI_network.pruned.interactions.binding.one_direction,
        CF_PPI_network.pruned.interactions.binding.other_direction)
CF_PPI_network.pruned.for_igraph <- rbind(CF_PPI_network.pruned.interactions.non_binding,
                                       CF_PPI_network.pruned.interactions.binding.both_directions)

In [13]:
CF_PPI_network.pruned.igraph <- graph_from_data_frame(CF_PPI_network.pruned.for_igraph, 
                                                   directed=TRUE)

## 1.5 Interesting nodes

### 1.5.1 Sink nodes

In [14]:
sink_nodes.final.pathways.df <- read.table(file = "../CFnetwork/data/sink_nodes/CFnetwork_sink_nodes_to_pathways.txt",
                                          sep = "\t",
                                          header = T,
                                          na.strings = "",
                                          check.names = FALSE)
sink_nodes.final.pathways.df <- sink_nodes.final.pathways.df[order(sink_nodes.final.pathways.df$Endpoint_cat),]
sink_nodes.final.pathways.df$Symbol <- as.character(sink_nodes.final.pathways.df$Symbol)
sink_nodes <- as.character(sink_nodes.final.pathways.df$Symbol)

### 1.5.2 Source nodes

In [15]:
source_nodes <- c("TRADD",
                  "PRKACA",
                  "SYK",
                  "CSNK2A1",
                  "SRC",
                  "PLCB1",
                  "PLCB3",
                  "EZR")

### 1.5.3 CFTR interactors

In [16]:
CFTR_interactors = CF_PPI_network.pruned.nodes[which(CF_PPI_network.pruned.nodes$CFTR_interactor),"Symbol"]

In [17]:
CFTR_interactors %in% CF_PPI_network.pruned@nodes$Symbol

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE

# 2. Differentially methylated CpG sites between CF and NCF dataframe

In [18]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.5
✔ ggplot2   3.5.1     ✔ stringr   1.5.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ lubridate::%--%()       masks igraph::%--%()
✖ tibble::as_data_frame() masks dplyr::as_data_frame(), igraph::as_data_frame()
✖ purrr::compose()        masks igraph::compose()
✖ tidyr::crossing()       masks igraph::crossing()
✖ dplyr::filter()         masks stats::filter()
✖ dplyr::lag()            masks stats::lag()
✖ purrr::simplify()       masks igraph::simplify()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [19]:
dm_CF_NCF_sup_table3_1267CpG <- 
  read.table(file = "../CFMethylomeSysbio/dm_CpG_CF_NCF_Magalhaes_2018_sup_table_3.tsv",
             skip = 1,
             quote = "\"",
             sep = "\t",
             header = T,
             check.names = F,
            nrows = 1267,
            na.strings = "-")

dm_CF_NCF_sup_table3_1267CpG <- 
  dm_CF_NCF_sup_table3_1267CpG[order(dm_CF_NCF_sup_table3_1267CpG$`p-value`),]
dm_CF_NCF_sup_table3_1267CpG$CpG_pvalue_rank_ <- 1:dim(dm_CF_NCF_sup_table3_1267CpG)[1]

In [20]:
colnames(dm_CF_NCF_sup_table3_1267CpG)[4:5] <- c("Median_CF_patients","Median_Controls")

In [21]:
dm_CF_NCF_sup_table3_1267CpG <- dm_CF_NCF_sup_table3_1267CpG %>%
    mutate(Median_CF_patients = as.numeric(gsub(pattern = ",", ".", Median_CF_patients, fixed = TRUE))) %>%
    mutate(Median_Controls = as.numeric(gsub(pattern = ",", ".", Median_Controls, fixed = TRUE)))

In [22]:
sapply(dm_CF_NCF_sup_table3_1267CpG, class)

Probe ID               Gene            p-value Median_CF_patients 
       "character"        "character"          "numeric"          "numeric" 
   Median_Controls   Genomic location      Island status          *Enhancer 
         "numeric"        "character"        "character"          "logical" 
  CpG_pvalue_rank_ 
         "integer"

In [23]:
dm_CF_NCF_sup_table3_1267CpG$abs_change <- abs(dm_CF_NCF_sup_table3_1267CpG$Median_CF_patients - dm_CF_NCF_sup_table3_1267CpG$Median_Controls)

In [61]:
mean(dm_CF_NCF_sup_table3_1267CpG$abs_change)

[1] 0.139116

In [24]:
dm_CF_NCF_sup_table3_1267CpG <- 
  dm_CF_NCF_sup_table3_1267CpG[order(dm_CF_NCF_sup_table3_1267CpG$abs_change, decreasing = T),]
dm_CF_NCF_sup_table3_1267CpG$CpG_beta_value_rank_ <- 1:dim(dm_CF_NCF_sup_table3_1267CpG)[1]

In [25]:
dm_CF_NCF_sup_table3_1267CpG <- 
  dm_CF_NCF_sup_table3_1267CpG[order(dm_CF_NCF_sup_table3_1267CpG$`p-value`),]

## 2.1 Keeping only CpG sites in the body of the genes

In [26]:
dm_CF_NCF_sup_table3_1267CpG_onlyGenes <- 
  dm_CF_NCF_sup_table3_1267CpG[which(dm_CF_NCF_sup_table3_1267CpG$`Genomic location`=="Body"),]

dm_CF_NCF_sup_table3_1267CpG_onlyGenes$Gene_rank <- 1:dim(dm_CF_NCF_sup_table3_1267CpG_onlyGenes)[1]

## 2.2 Searching for dm genes in the CF network

In [27]:
dm_CF_NCF_sup_table3_1267CpG_Genes.list <- unique(dm_CF_NCF_sup_table3_1267CpG_onlyGenes$Gene)

dm_CF_NCF_Genes_in_CF_network.list <- 
  dm_CF_NCF_sup_table3_1267CpG_Genes.list[which(dm_CF_NCF_sup_table3_1267CpG_Genes.list %in% CF_PPI_network.pruned.nodes$Symbol)]

dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df <-
  dm_CF_NCF_sup_table3_1267CpG[which(dm_CF_NCF_sup_table3_1267CpG$Gene %in% dm_CF_NCF_Genes_in_CF_network.list),]

In [28]:
dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df

,Probe ID,Gene,p-value,Median_CF_patients,Median_Controls,Genomic location,Island status,*Enhancer,CpG_pvalue_rank_,abs_change,CpG_beta_value_rank_
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<lgl>,<int>,<dbl>,<int>
647,cg13433729,EZR,1.2e-06,0.61,0.49,Body,Open sea,TRUE,2,0.12,702
104,cg01899581,DIAPH3,9.0e-05,0.64,0.47,Body,N_Shelf,FALSE,53,0.17,184
552,cg11321921,NFATC1,5.5e-04,0.78,0.91,Body,Island,FALSE,240,0.13,562
306,cg05687091,NFATC1,6.1e-04,0.54,0.64,Body,Island,FALSE,257,0.10,1196
1086,cg23752651,TNFRSF1A,9.5e-04,0.58,0.70,Body,Open sea,FALSE,374,0.12,752
1007,cg22240998,ITPR1,1.1e-03,0.60,0.50,Body,Open sea,TRUE,420,0.10,1205
891,cg18984715,CBLB,1.2e-03,0.79,0.67,Body,Open sea,TRUE,441,0.12,762
234,cg04221461,AKT3,2.2e-03,0.46,0.56,Body,Open sea,FALSE,613,0.10,1158
968,cg21070009,MAP2K1,4.1e-03,0.48,0.35,Body,Open sea,TRUE,852,0.13,626


## 2.3 Network analysis

In [29]:
CF_PPI_network.pruned.dists <- distances(CF_PPI_network.pruned.igraph, mode = "out")

### 2.3.1 Distance to sink nodes

In [30]:
### DM gene to sink nodes
CF_PPI_network.pruned.dm_to_sink_nodes <- CF_PPI_network.pruned.dists[dm_CF_NCF_Genes_in_CF_network.list, sink_nodes]


In [31]:
CF_PPI_network.pruned.dm_to_sink_nodes

,CASP1,CASP3,CASP7,CYBA,CYBB,DNM1L,GABARAP,ACTN4,ARPC5,CFL1,⋯,IRF5,IRF7,IRF9,JUN,NFATC1,NFKB1,NFKB2,RELA,RELB,STAT1
EZR,Inf,Inf,Inf,Inf,Inf,Inf,Inf,7,Inf,7,⋯,Inf,Inf,Inf,5,Inf,4,5,3,4,Inf
DIAPH3,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,3,Inf,⋯,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf
NFATC1,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,⋯,Inf,Inf,Inf,Inf,0,Inf,Inf,Inf,Inf,Inf
TNFRSF1A,4,3,4,5,6,4,9,6,7,7,⋯,4,4,5,5,5,4,4,5,5,4
ITPR1,3,4,4,10,11,7,6,10,12,10,⋯,5,5,8,6,3,4,7,5,6,7
CBLB,6,5,6,5,6,6,9,6,7,6,⋯,5,5,5,3,4,4,5,5,6,4
AKT3,7,7,7,9,10,8,9,6,11,6,⋯,6,2,6,4,2,3,2,4,3,5
MAP2K1,Inf,Inf,Inf,Inf,Inf,Inf,Inf,4,Inf,4,⋯,Inf,Inf,Inf,2,Inf,8,9,7,8,Inf
PLCG2,4,5,5,6,7,6,7,5,8,5,⋯,6,5,6,3,2,4,5,5,6,5
BAIAP2,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,2,Inf,⋯,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf


In [32]:
sink_nodes_downstream_to_dm <- data.frame(apply(X = CF_PPI_network.pruned.dm_to_sink_nodes,
      MARGIN = 1,
      FUN=function(x){
        return(sum(!is.infinite(x)))
      }))
colnames(sink_nodes_downstream_to_dm) <- "dist_to_CF_network_sink_nodes"
sink_nodes_downstream_to_dm['Gene'] <- rownames(sink_nodes_downstream_to_dm)
rownames(sink_nodes_downstream_to_dm) <- NULL

In [33]:
dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df <- merge(dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df,
                                                    sink_nodes_downstream_to_dm,
                                                    by = "Gene")

In [34]:
dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df

Gene,Probe ID,p-value,Median_CF_patients,Median_Controls,Genomic location,Island status,*Enhancer,CpG_pvalue_rank_,abs_change,CpG_beta_value_rank_,dist_to_CF_network_sink_nodes
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<lgl>,<int>,<dbl>,<int>,<int>
AKT3,cg04221461,2.2e-03,0.46,0.56,Body,Open sea,FALSE,613,0.10,1158,35
BAIAP2,cg02026049,7.9e-03,0.65,0.76,Body,N_Shelf,FALSE,1123,0.11,1108,2
CBLB,cg18984715,1.2e-03,0.79,0.67,Body,Open sea,TRUE,441,0.12,762,35
DIAPH3,cg01899581,9.0e-05,0.64,0.47,Body,N_Shelf,FALSE,53,0.17,184,2
EZR,cg13433729,1.2e-06,0.61,0.49,Body,Open sea,TRUE,2,0.12,702,15
ITPR1,cg22240998,1.1e-03,0.60,0.50,Body,Open sea,TRUE,420,0.10,1205,35
MAP2K1,cg21070009,4.1e-03,0.48,0.35,Body,Open sea,TRUE,852,0.13,626,15
NFATC1,cg11321921,5.5e-04,0.78,0.91,Body,Island,FALSE,240,0.13,562,1
NFATC1,cg05687091,6.1e-04,0.54,0.64,Body,Island,FALSE,257,0.10,1196,1


### 2.3.2 Distance to CFTR interactors

In [35]:
### CFTR interactors to diff methylated
CF_PPI_network.pruned.dm_from_CFTR_interactors <- data.frame(t(CF_PPI_network.pruned.dists[CFTR_interactors, dm_CF_NCF_Genes_in_CF_network.list]))
colnames(CF_PPI_network.pruned.dm_from_CFTR_interactors) <- paste("from_",
                                                              colnames(CF_PPI_network.pruned.dm_from_CFTR_interactors),
                                                              sep = "")

In [36]:
CF_PPI_network.pruned.dm_from_CFTR_interactors

,from_ADORA2B,from_CAPN1,from_CAPNS1,from_CAV1,from_CAV2,from_CSNK2A1,from_PRKACA,from_PRKCE,from_SLC9A3R1,from_SLC9A3R2,from_SYK,from_TRADD
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
EZR,1,1,1,Inf,Inf,Inf,6,1,Inf,Inf,6,7
DIAPH3,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,9,8
NFATC1,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,3,5
TNFRSF1A,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf
ITPR1,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,2,4
CBLB,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf
AKT3,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,5,3
MAP2K1,4,4,4,Inf,Inf,Inf,2,4,Inf,Inf,4,3
PLCG2,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,1,3


In [37]:
CF_PPI_network.pruned.dm_from_CFTR_interactors['Gene'] <- rownames(CF_PPI_network.pruned.dm_from_CFTR_interactors)
rownames(CF_PPI_network.pruned.dm_from_CFTR_interactors) <- NULL
CF_PPI_network.pruned.dm_from_CFTR_interactors

from_ADORA2B,from_CAPN1,from_CAPNS1,from_CAV1,from_CAV2,from_CSNK2A1,from_PRKACA,from_PRKCE,from_SLC9A3R1,from_SLC9A3R2,from_SYK,from_TRADD,Gene
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,1,1,Inf,Inf,Inf,6,1,Inf,Inf,6,7,EZR
Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,9,8,DIAPH3
Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,3,5,NFATC1
Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,TNFRSF1A
Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,2,4,ITPR1
Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,CBLB
Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,5,3,AKT3
4,4,4,Inf,Inf,Inf,2,4,Inf,Inf,4,3,MAP2K1
Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,1,3,PLCG2


In [38]:
dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df <- merge(dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df,
                                                    CF_PPI_network.pruned.dm_from_CFTR_interactors,
                                                    by = "Gene")
dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df

Gene,Probe ID,p-value,Median_CF_patients,Median_Controls,Genomic location,Island status,*Enhancer,CpG_pvalue_rank_,abs_change,⋯,from_CAPNS1,from_CAV1,from_CAV2,from_CSNK2A1,from_PRKACA,from_PRKCE,from_SLC9A3R1,from_SLC9A3R2,from_SYK,from_TRADD
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<lgl>,<int>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AKT3,cg04221461,2.2e-03,0.46,0.56,Body,Open sea,FALSE,613,0.10,⋯,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,5,3
BAIAP2,cg02026049,7.9e-03,0.65,0.76,Body,N_Shelf,FALSE,1123,0.11,⋯,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,6,5
CBLB,cg18984715,1.2e-03,0.79,0.67,Body,Open sea,TRUE,441,0.12,⋯,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf
DIAPH3,cg01899581,9.0e-05,0.64,0.47,Body,N_Shelf,FALSE,53,0.17,⋯,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,9,8
EZR,cg13433729,1.2e-06,0.61,0.49,Body,Open sea,TRUE,2,0.12,⋯,1,Inf,Inf,Inf,6,1,Inf,Inf,6,7
ITPR1,cg22240998,1.1e-03,0.60,0.50,Body,Open sea,TRUE,420,0.10,⋯,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,2,4
MAP2K1,cg21070009,4.1e-03,0.48,0.35,Body,Open sea,TRUE,852,0.13,⋯,4,Inf,Inf,Inf,2,4,Inf,Inf,4,3
NFATC1,cg11321921,5.5e-04,0.78,0.91,Body,Island,FALSE,240,0.13,⋯,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,3,5
NFATC1,cg05687091,6.1e-04,0.54,0.64,Body,Island,FALSE,257,0.10,⋯,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,3,5


In [39]:
colnames(CF_PPI_network.pruned.dm_from_CFTR_interactors)

[1] "from_ADORA2B"  "from_CAPN1"    "from_CAPNS1"   "from_CAV1"    
 [5] "from_CAV2"     "from_CSNK2A1"  "from_PRKACA"   "from_PRKCE"   
 [9] "from_SLC9A3R1" "from_SLC9A3R2" "from_SYK"      "from_TRADD"   
[13] "Gene"

In [40]:
dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df$min_distance_to_CFTR_interactor <- apply(dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df[,colnames(CF_PPI_network.pruned.dm_from_CFTR_interactors)],
      1,
       FUN = function(x) {return(min(x, finite=T))})

In [41]:
dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df

Gene,Probe ID,p-value,Median_CF_patients,Median_Controls,Genomic location,Island status,*Enhancer,CpG_pvalue_rank_,abs_change,⋯,from_CAV1,from_CAV2,from_CSNK2A1,from_PRKACA,from_PRKCE,from_SLC9A3R1,from_SLC9A3R2,from_SYK,from_TRADD,min_distance_to_CFTR_interactor
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<lgl>,<int>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
AKT3,cg04221461,2.2e-03,0.46,0.56,Body,Open sea,FALSE,613,0.10,⋯,Inf,Inf,Inf,Inf,Inf,Inf,Inf,5,3,3
BAIAP2,cg02026049,7.9e-03,0.65,0.76,Body,N_Shelf,FALSE,1123,0.11,⋯,Inf,Inf,Inf,Inf,Inf,Inf,Inf,6,5,5
CBLB,cg18984715,1.2e-03,0.79,0.67,Body,Open sea,TRUE,441,0.12,⋯,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,1
DIAPH3,cg01899581,9.0e-05,0.64,0.47,Body,N_Shelf,FALSE,53,0.17,⋯,Inf,Inf,Inf,Inf,Inf,Inf,Inf,9,8,8
EZR,cg13433729,1.2e-06,0.61,0.49,Body,Open sea,TRUE,2,0.12,⋯,Inf,Inf,Inf,6,1,Inf,Inf,6,7,1
ITPR1,cg22240998,1.1e-03,0.60,0.50,Body,Open sea,TRUE,420,0.10,⋯,Inf,Inf,Inf,Inf,Inf,Inf,Inf,2,4,2
MAP2K1,cg21070009,4.1e-03,0.48,0.35,Body,Open sea,TRUE,852,0.13,⋯,Inf,Inf,Inf,2,4,Inf,Inf,4,3,2
NFATC1,cg11321921,5.5e-04,0.78,0.91,Body,Island,FALSE,240,0.13,⋯,Inf,Inf,Inf,Inf,Inf,Inf,Inf,3,5,3
NFATC1,cg05687091,6.1e-04,0.54,0.64,Body,Island,FALSE,257,0.10,⋯,Inf,Inf,Inf,Inf,Inf,Inf,Inf,3,5,3


### 2.3.2 Distance to source nodes

In [44]:
source_nodes_not_CFTR_interactors <- source_nodes[!source_nodes %in% CFTR_interactors]

In [45]:
source_nodes_not_CFTR_interactors

[1] "SRC"   "PLCB1" "PLCB3" "EZR"

In [46]:
### Source nodes to diff methylated
CF_PPI_network.pruned.dm_from_source_nodes <- data.frame(t(CF_PPI_network.pruned.dists[source_nodes_not_CFTR_interactors, dm_CF_NCF_Genes_in_CF_network.list]))
colnames(CF_PPI_network.pruned.dm_from_source_nodes) <- paste("from_",
                                                              colnames(CF_PPI_network.pruned.dm_from_source_nodes),
                                                              sep = "")

In [47]:
rownames(CF_PPI_network.pruned.dm_from_source_nodes)

[1] "EZR"      "DIAPH3"   "NFATC1"   "TNFRSF1A" "ITPR1"    "CBLB"    
 [7] "AKT3"     "MAP2K1"   "PLCG2"    "BAIAP2"

In [48]:
CF_PPI_network.pruned.dm_from_source_nodes['Gene'] <- rownames(CF_PPI_network.pruned.dm_from_source_nodes)
rownames(CF_PPI_network.pruned.dm_from_source_nodes) <- NULL

In [49]:
dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df <- merge(dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df,
                                                    CF_PPI_network.pruned.dm_from_source_nodes,
                                                    by = "Gene")
dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df

Gene,Probe ID,p-value,Median_CF_patients,Median_Controls,Genomic location,Island status,*Enhancer,CpG_pvalue_rank_,abs_change,⋯,from_PRKCE,from_SLC9A3R1,from_SLC9A3R2,from_SYK,from_TRADD,min_distance_to_CFTR_interactor,from_SRC,from_PLCB1,from_PLCB3,from_EZR
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<lgl>,<int>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
AKT3,cg04221461,2.2e-03,0.46,0.56,Body,Open sea,FALSE,613,0.10,⋯,Inf,Inf,Inf,5,3,3,2,7,7,Inf
BAIAP2,cg02026049,7.9e-03,0.65,0.76,Body,N_Shelf,FALSE,1123,0.11,⋯,Inf,Inf,Inf,6,5,5,4,9,9,Inf
CBLB,cg18984715,1.2e-03,0.79,0.67,Body,Open sea,TRUE,441,0.12,⋯,Inf,Inf,Inf,Inf,Inf,1,Inf,Inf,Inf,Inf
DIAPH3,cg01899581,9.0e-05,0.64,0.47,Body,N_Shelf,FALSE,53,0.17,⋯,Inf,Inf,Inf,9,8,8,7,12,12,Inf
EZR,cg13433729,1.2e-06,0.61,0.49,Body,Open sea,TRUE,2,0.12,⋯,1,Inf,Inf,6,7,1,5,5,5,0
ITPR1,cg22240998,1.1e-03,0.60,0.50,Body,Open sea,TRUE,420,0.10,⋯,Inf,Inf,Inf,2,4,2,10,1,1,Inf
MAP2K1,cg21070009,4.1e-03,0.48,0.35,Body,Open sea,TRUE,852,0.13,⋯,4,Inf,Inf,4,3,2,2,5,5,3
NFATC1,cg11321921,5.5e-04,0.78,0.91,Body,Island,FALSE,240,0.13,⋯,Inf,Inf,Inf,3,5,3,4,4,4,Inf
NFATC1,cg05687091,6.1e-04,0.54,0.64,Body,Island,FALSE,257,0.10,⋯,Inf,Inf,Inf,3,5,3,4,4,4,Inf


In [50]:
dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df$min_distance_to_source_nodes_2 <- apply(dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df[,c(colnames(CF_PPI_network.pruned.dm_from_source_nodes),
                                                                                                                               "min_distance_to_CFTR_interactor")],
      1,
       FUN = function(x) {return(min(x, finite=T))})

In [51]:
dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df

Gene,Probe ID,p-value,Median_CF_patients,Median_Controls,Genomic location,Island status,*Enhancer,CpG_pvalue_rank_,abs_change,⋯,from_SLC9A3R1,from_SLC9A3R2,from_SYK,from_TRADD,min_distance_to_CFTR_interactor,from_SRC,from_PLCB1,from_PLCB3,from_EZR,min_distance_to_source_nodes_2
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<lgl>,<int>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
AKT3,cg04221461,2.2e-03,0.46,0.56,Body,Open sea,FALSE,613,0.10,⋯,Inf,Inf,5,3,3,2,7,7,Inf,2
BAIAP2,cg02026049,7.9e-03,0.65,0.76,Body,N_Shelf,FALSE,1123,0.11,⋯,Inf,Inf,6,5,5,4,9,9,Inf,4
CBLB,cg18984715,1.2e-03,0.79,0.67,Body,Open sea,TRUE,441,0.12,⋯,Inf,Inf,Inf,Inf,1,Inf,Inf,Inf,Inf,1
DIAPH3,cg01899581,9.0e-05,0.64,0.47,Body,N_Shelf,FALSE,53,0.17,⋯,Inf,Inf,9,8,8,7,12,12,Inf,7
EZR,cg13433729,1.2e-06,0.61,0.49,Body,Open sea,TRUE,2,0.12,⋯,Inf,Inf,6,7,1,5,5,5,0,0
ITPR1,cg22240998,1.1e-03,0.60,0.50,Body,Open sea,TRUE,420,0.10,⋯,Inf,Inf,2,4,2,10,1,1,Inf,1
MAP2K1,cg21070009,4.1e-03,0.48,0.35,Body,Open sea,TRUE,852,0.13,⋯,Inf,Inf,4,3,2,2,5,5,3,2
NFATC1,cg11321921,5.5e-04,0.78,0.91,Body,Island,FALSE,240,0.13,⋯,Inf,Inf,3,5,3,4,4,4,Inf,3
NFATC1,cg05687091,6.1e-04,0.54,0.64,Body,Island,FALSE,257,0.10,⋯,Inf,Inf,3,5,3,4,4,4,Inf,3


### 2.3.4 Betweenness centrality

In [52]:
CF_PPI_network.bc.df <- data.frame(betweenness(CF_PPI_network.pruned.igraph))
CF_PPI_network.bc.df$Symbol <- rownames(CF_PPI_network.bc.df)
rownames(CF_PPI_network.bc.df) <- NULL
colnames(CF_PPI_network.bc.df) <- c("BC.score", "Symbol")

In [53]:
CF_PPI_network.bc.df

BC.score,Symbol
<dbl>,<chr>
93.0000000,PLCB3
383.0000000,TLR4
8.0000000,WASL
744.0440476,MAP2K7
0.0000000,TNFSF11
663.0399625,IKBKB
0.0000000,PPP1R12A
1620.7172382,CHUK
0.0000000,TLR3


In [54]:
CF_PPI_network.bc.df.dm_CF_NCF_df <- CF_PPI_network.bc.df[which(CF_PPI_network.bc.df$Symbol %in%  dm_CF_NCF_Genes_in_CF_network.list),]

In [55]:
CF_PPI_network.bc.df.dm_CF_NCF_df

,BC.score,Symbol
,<dbl>,<chr>
69,3295.1806,PLCG2
73,419.6667,TNFRSF1A
120,2361.4579,MAP2K1
137,0.0000,CBLB
149,602.8583,ITPR1
214,15.0000,DIAPH3
225,661.0000,BAIAP2
227,2784.6197,AKT3
238,747.3944,EZR


In [56]:
dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df <- merge(dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df,
                                                    CF_PPI_network.bc.df.dm_CF_NCF_df,
                                                    by.x = "Gene",
                                                    by.y = "Symbol",
                                                   all.x = T)
dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df

Gene,Probe ID,p-value,Median_CF_patients,Median_Controls,Genomic location,Island status,*Enhancer,CpG_pvalue_rank_,abs_change,⋯,from_SLC9A3R2,from_SYK,from_TRADD,min_distance_to_CFTR_interactor,from_SRC,from_PLCB1,from_PLCB3,from_EZR,min_distance_to_source_nodes_2,BC.score
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<lgl>,<int>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
AKT3,cg04221461,2.2e-03,0.46,0.56,Body,Open sea,FALSE,613,0.10,⋯,Inf,5,3,3,2,7,7,Inf,2,2784.6197
BAIAP2,cg02026049,7.9e-03,0.65,0.76,Body,N_Shelf,FALSE,1123,0.11,⋯,Inf,6,5,5,4,9,9,Inf,4,661.0000
CBLB,cg18984715,1.2e-03,0.79,0.67,Body,Open sea,TRUE,441,0.12,⋯,Inf,Inf,Inf,1,Inf,Inf,Inf,Inf,1,0.0000
DIAPH3,cg01899581,9.0e-05,0.64,0.47,Body,N_Shelf,FALSE,53,0.17,⋯,Inf,9,8,8,7,12,12,Inf,7,15.0000
EZR,cg13433729,1.2e-06,0.61,0.49,Body,Open sea,TRUE,2,0.12,⋯,Inf,6,7,1,5,5,5,0,0,747.3944
ITPR1,cg22240998,1.1e-03,0.60,0.50,Body,Open sea,TRUE,420,0.10,⋯,Inf,2,4,2,10,1,1,Inf,1,602.8583
MAP2K1,cg21070009,4.1e-03,0.48,0.35,Body,Open sea,TRUE,852,0.13,⋯,Inf,4,3,2,2,5,5,3,2,2361.4579
NFATC1,cg11321921,5.5e-04,0.78,0.91,Body,Island,FALSE,240,0.13,⋯,Inf,3,5,3,4,4,4,Inf,3,0.0000
NFATC1,cg05687091,6.1e-04,0.54,0.64,Body,Island,FALSE,257,0.10,⋯,Inf,3,5,3,4,4,4,Inf,3,0.0000


### 2.3.5 Pathways

In [57]:
dm_CF_NCF_Genes_in_CF_network.CF_network_df <- CF_PPI_network.pruned.nodes[which(CF_PPI_network.pruned.nodes$Symbol %in% dm_CF_NCF_Genes_in_CF_network.list),]

dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df <- merge(dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df,
                                                    CF_PPI_network.pruned.nodes[,1:18],
                                                    by.x = "Gene",
                                                    by.y = "Symbol",
                                                   all.x = T)
dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df

Gene,Probe ID,p-value,Median_CF_patients,Median_Controls,Genomic location,Island status,*Enhancer,CpG_pvalue_rank_,abs_change,⋯,Osteoclast differentiation,Regulation of actin cytoskeleton,RIG-I-like receptor signaling pathway,T cell receptor signaling pathway,Th17 cell differentiation,TNF signaling pathway,Toll-like receptor signaling pathway,Viral protein interaction with cytokine and cytokine receptor,sum,pathway
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<lgl>,<int>,<dbl>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>
AKT3,cg04221461,2.2e-03,0.46,0.56,Body,Open sea,FALSE,613,0.10,⋯,1,0,0,1,0,1,1,0,6,NA
BAIAP2,cg02026049,7.9e-03,0.65,0.76,Body,N_Shelf,FALSE,1123,0.11,⋯,0,1,0,0,0,0,0,0,1,Regulation of actin cytoskeleton
CBLB,cg18984715,1.2e-03,0.79,0.67,Body,Open sea,TRUE,441,0.12,⋯,0,0,0,1,0,0,0,0,2,NA
DIAPH3,cg01899581,9.0e-05,0.64,0.47,Body,N_Shelf,FALSE,53,0.17,⋯,0,1,0,0,0,0,0,0,1,Regulation of actin cytoskeleton
EZR,cg13433729,1.2e-06,0.61,0.49,Body,Open sea,TRUE,2,0.12,⋯,0,1,0,0,0,0,0,0,1,Regulation of actin cytoskeleton
ITPR1,cg22240998,1.1e-03,0.60,0.50,Body,Open sea,TRUE,420,0.10,⋯,0,0,0,0,0,0,0,0,3,NA
MAP2K1,cg21070009,4.1e-03,0.48,0.35,Body,Open sea,TRUE,852,0.13,⋯,1,1,0,1,0,1,1,0,6,NA
NFATC1,cg11321921,5.5e-04,0.78,0.91,Body,Island,FALSE,240,0.13,⋯,1,0,0,1,1,0,0,0,4,NA
NFATC1,cg05687091,6.1e-04,0.54,0.64,Body,Island,FALSE,257,0.10,⋯,1,0,0,1,1,0,0,0,4,NA


In [58]:
write.table(dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df,
            file = "../CFMethylomeSysbio/dm_CpG_CF_NCF_in_CF_network_analysis_20241009.tsv",
            sep = "\t",
            row.names = F,
            quote = FALSE)

## Adding a tag in the CF network

In [60]:
CF_PPI_network.pruned.nodes$diff_methylated <- CF_PPI_network.pruned.nodes$Symbol %in% dm_CF_NCF_Genes_in_CF_network.list



FALSE  TRUE 
  329    10 

In [62]:
colnames(CF_PPI_network.pruned.nodes)

[1] "Symbol"                                                       
 [2] "C-type lectin receptor signaling pathway"                     
 [3] "Cytokine-cytokine receptor interaction"                       
 [4] "Cytosolic DNA-sensing pathway"                                
 [5] "Estrogen signaling pathway"                                   
 [6] "IL-17 signaling pathway"                                      
 [7] "NF-kappa B signaling pathway"                                 
 [8] "NOD-like receptor signaling pathway"                          
 [9] "Osteoclast differentiation"                                   
[10] "Regulation of actin cytoskeleton"                             
[11] "RIG-I-like receptor signaling pathway"                        
[12] "T cell receptor signaling pathway"                            
[13] "Th17 cell differentiation"                                    
[14] "TNF signaling pathway"                                        
[15] "Toll-like receptor signaling pathway"                         
[16] "Viral protein interaction with cytokine and cytokine receptor"
[17] "sum"                                                          
[18] "pathway"                                                      
[19] "CFTR_interactor"                                              
[20] "dorothea_tf"                                                  
[21] "receptor_ligand"                                              
[22] "receptor"                                                     
[23] "diff_methylated"

# 3. Differentially methylated CpG sites between severe and mild dataframe

In [ ]:
dm_severe_mild_sup_table4_189CpG <- 
  read.table(file = "../CFMethylomeSysbio/dm_CpG_CF_severe_mild_Magalhaes_2018_sup_table_4.tsv",
             skip = 1,
             quote = "\"",
             sep = "\t",
             header = T,
             check.names = F)

dm_severe_mild_sup_table4_189CpG <- 
  dm_severe_mild_sup_table4_189CpG[order(dm_severe_mild_sup_table4_189CpG$`p-value`),]
dm_severe_mild_sup_table4_189CpG$CpG_rank <- 1:dim(dm_severe_mild_sup_table4_189CpG)[1]

## 3.1 Keeping only CpG sites in the body of the genes

In [ ]:
dm_severe_mild_sup_table4_189CpG_onlyGenes <- 
  dm_severe_mild_sup_table4_189CpG[which(dm_severe_mild_sup_table4_189CpG$`Genomic location`=="Body"),]

dm_severe_mild_sup_table4_189CpG_onlyGenes$Gene_rank <- 1:dim(dm_severe_mild_sup_table4_189CpG_onlyGenes)[1]

# 3.2 Searching for dm genes in the CF network

In [ ]:
dm_severe_mild_sup_table4_189CpG_Genes.list <- unique(dm_severe_mild_sup_table4_189CpG_onlyGenes$Gene)

dm_CF_NCF_Genes_in_CF_network.list <- 
  dm_severe_mild_sup_table4_189CpG_Genes.list[which(dm_severe_mild_sup_table4_189CpG_Genes.list %in% CF_PPI_network.pruned.nodes$Symbol)]

dm_CF_NCF_Genes_in_CF_network.CF_network_df <- 
  CF_PPI_network.lcc.node_type.nodes[which(CF_PPI_network.pruned.nodes$Symbol %in% dm_CF_NCF_Genes_in_CF_network.list),]

dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df <-
  dm_severe_mild_sup_table4_189CpG[which(dm_severe_mild_sup_table4_189CpG$Gene %in% dm_CF_NCF_Genes_in_CF_network.list),]

In [ ]:
dm_CF_NCF_Genes_in_CF_network.list